# Species Analysis

## import

In [1]:
import os

In [2]:
import re

In [3]:
import pandas as pd

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
import json

In [7]:
%run ../../Util/FileFolder

created


In [8]:
%run ../../Util/Variable

# subfolders

In [9]:
subfolder={}

## sub folder count

In [10]:
subfolderCount=num_sub_folders(original_dataset_dir)
subfolder['count']=subfolderCount

In [11]:
today_collection_path=os.path.join(today_summary_path,'collection')


In [12]:
overviewTxt=os.path.join(today_collection_path,'Overview.txt')

In [13]:
with open (overviewTxt,'a') as overviewFile:
    overviewFile.write(f'species:{subfolderCount}\n')

### Subfolder names


In [14]:
folders_names=sub_folders(original_dataset_dir)

In [15]:
subfoldersNamesTxt=os.path.join(today_collection_path,'subfolders_Names.txt')

In [16]:
with open (subfoldersNamesTxt,'a') as subfoldersFile:
    for subfolder in folders_names:
        subfoldersFile.write(f'{subfolder}\n')

In [17]:
folders=list(
    map(
        lambda name: 
            {'name':name,
             'path':os.path.join(original_dataset_dir,name),
             'filecount':count_files_in_folder(os.path.join(original_dataset_dir,name)),
             'volume':{
                 'GB':folder_size_in_gb(os.path.join(original_dataset_dir,name)),
                 "KB":folder_size_in_kb(os.path.join(original_dataset_dir,name)),
                "MB":folder_size_in_mb(os.path.join(original_dataset_dir,name))
             }
            },folders_names))
folder_li=list(map( lambda name: 
            [name,
             folder_size_in_gb(os.path.join(original_dataset_dir,name)),
            folder_size_in_kb(os.path.join(original_dataset_dir,name)),
            folder_size_in_mb(os.path.join(original_dataset_dir,name))
             
            ],folders_names))
folder_count_li=list(map( lambda name: 
            [name,
            count_files_in_folder(os.path.join(original_dataset_dir,name)),
             
            ],folders_names))

In [18]:
save_to_csv_filedetails(data=folder_li,colums=['Names','VolumeKB','VolumeMB',"VolumeGB"],csv_filename='volumereportcsv.csv',save_folder=today_collection_path)

32

In [19]:
save_to_csv_filedetails(data=folder_count_li,colums=['Names','counts'],csv_filename='countreportcsv.csv',save_folder=today_collection_path)

32

In [20]:
subfolderpaths=subfolders_path(original_dataset_dir)

In [21]:
subfoldersPathsTxt=os.path.join(today_collection_path,'subfolders_paths.txt')

In [22]:
with open (subfoldersPathsTxt,'a') as subfoldersFile:
    for subfolder in subfolderpaths:
        subfoldersFile.write(f'{subfolder}\n')

### subfolder details

In [23]:
subfolders=[]

In [24]:
for subfolder in folders:
     subfoldernames=sub_folders(subfolder['path'])
     subchild=list(
        map(lambda name: {
                'name':name,
                'path':os.path.join(subfolder['path'],name),
                 'filecount':count_files_in_folder(os.path.join(subfolder['path'],name)),
                 'volume':{
                     'GB':folder_size_in_gb(os.path.join(subfolder['path'],name)),
                     "KB":folder_size_in_kb(os.path.join(subfolder['path'],name)),
                    "MB":folder_size_in_mb(os.path.join(subfolder['path'],name)),
                 },
                },subfoldernames)),
     subfolder['subfolders']=subchild,
     subfolders.append(subfolder)
     
     
        

In [25]:
leafpattern = re.compile(r'^(leaf|leafs|leaves|Leaves)$')
plantpattern=re.compile(r'^(Plants|plant|p|Plant)$')
rawFruitpattern=re.compile(r'^(fruits|Fruit|kaai|Fruits,raw_fruits,seeds)$')
flowerpattern=re.compile(r'^(fruits|fruit|Fruits)$')

In [26]:
subsub=[]
for i,subpath in enumerate(subfolderpaths):
    subsubnames=sub_folders(subpath)
    subsubpath=subfolders_path(subpath)
    sub={
        'speciesname':folders_names[i],
    }
    for ind,s in enumerate(subsubnames):
        sub['path']=subsubpath[ind]
        if leafpattern.match(s):
            sub['leaves']=num_sub_folders(subsubpath[ind])
        elif plantpattern.match(s):
            sub['plants']=num_sub_folders(subsubpath[ind])
        elif flowerpattern.match(s):
            sub['flowers']=num_sub_folders(subsubpath[ind])
        elif rawFruitpattern.match(s):
            sub['raw_fruits']=num_sub_folders(subsubpath[ind])
        else:
             sub[s]=num_sub_folders(subsubpath[ind])
    subsub.append(sub)

In [27]:
def subpartcountmaping(subpart):
    subpartRow=[]
    subpartRow.append(subpart['speciesname'])
    for part in ['leaves','plants','flowers','raw_fruits']:
        subpartRow.append(subpart[part]) if part in subpart.keys() else subpartRow.append(0)
    return subpartRow

In [28]:
subsubli=list(map(lambda tr:subpartcountmaping(tr),subsub))

In [29]:
reportjsonfile=os.path.join(today_collection_path,'reportjsonfile.json')

In [30]:
with open(reportjsonfile, 'r+') as f:
        # Load existing data from JSON
        existing_data = json.load(f)
        
        # Append new data to existing JSON data
        existing_data['subfolder']=subfolders
        
        # Write updated data back to JSON file
        f.seek(0)
        json.dump(existing_data, f, indent=4)
        f.truncate()

In [31]:
subfolderdetails =list(map(lambda folder: {
    'name':folder['name'],
    'path':folder['path'],
    'files':folder['filecount'],
    'volume(MB)':folder['volume']['MB'],
    },subfolders)) 
subfolderdetails_li=list(map(lambda folder: [
    folder['name'],
    folder['path'],
    folder['filecount'],
    folder['volume']['MB'],
    len(folder['subfolders'])
    ],subfolders)) 

In [32]:
save_to_csv_filedetails(data=subfolderdetails_li,colums=["Speciesname","Path","Leaves","Plants","Flowers","rawFruits"],csv_filename='subpart_reportcsv.csv',save_folder=today_collection_path)

32

In [33]:
save_to_csv_filedetails(data=subfolderdetails_li,colums=["Names","Path","Files","Volume MB","subfolders"],csv_filename='reportcsv.csv',save_folder=today_collection_path)

32